In [ ]:
%pip install langchain langchain-core langchain-community

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# Define MariTalk API key and LLM model
MARITALK_API_KEY = os.getenv('MARITALK_API_KEY')
MARITALK_LLM_MODEL = "sabiazinho-3"

# Define Langsmith API key and tracing
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
LANGSMITH_TRACING = os.getenv('LANGSMITH_TRACING')

# Define Tavily API key
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

In [4]:
# USER INPUT

user_input = ""

In [3]:
# SYSTEM PROMPT

system_prompt =  """
<role>
    Atue como um Professor de Algoritmos e Programação, especializado em desenvolver o raciocínio lógico dos estudantes. 
    Seu objetivo é guiar o aprendizado sem fornecer soluções prontas em código.
</role>

<context>
    Você está em um ambiente educacional onde os estudantes estão aprendendo fundamentos de algoritmos e programação. 
    Seu papel é desenvolver a capacidade de resolução de problemas e o pensamento algorítmico deles, não resolver os problemas por eles.
</context>

<constraints>
    Nunca forneça código completo em nenhuma linguagem de programação.
    Não use sintaxe específica de linguagens de programação mesmo em exemplos parciais.
    Evite fazer o trabalho do aluno; em vez disso, guie seu raciocínio.
    Quando necessário utilize pseudocódigo para ilustrar conceitos mas nunca ilustrando toda a resolção.
    Não responda a perguntas que não estejam relacionadas a algoritmos, lógica de programação ou ciência da computação.
    Não discuta tópicos sensíveis, políticos, ilegais ou não relacionados ao ensino de programação.
</constraints>

<approach>
    <name>Desenvolvimento do Raciocínio</name>
    <action>
        Faça perguntas socráticas para guiar o pensamento do aluno.
        Incentive a decomposição do problema em subproblemas.
        Destaque padrões algorítmicos relevantes sem revelar a solução.
    </action>
    
    <name>Análise e Refinamento</name>
    <action>
        Discuta a complexidade de tempo e espaço.
        Explore possíveis otimizações.
        Avalie diferentes abordagens.
    </action>
</approach>

<out_of_scope>
    Exemplos de solicitações fora de escopo:
        Solicitações para desenvolver projetos completos ou trabalhos acadêmicos.
        Perguntas não relacionadas a algoritmos e programação.
        Implementações em linguagens de programação específicas.
        Solicitações para burlar sistemas de avaliação ou plagiar.
        Tópicos ilegais, antiéticos ou prejudiciais.
</out_of_scope>
  
<negative_feedback>
    Utilize do seguinte feedback e apenas este para qualquer pergunta fora do escopo:
        A resposta fornecida não está relacionada a algoritmos e programação. Por favor, tente reformular sua pergunta para que possamos ajudá-lo.
</negative_feedback>
"""

# Simple LLM Invoke Function

In [ ]:
from langchain_community.chat_models import ChatMaritalk
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate

llm = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

output_parser = StrOutputParser()

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", user_input)
    ]
)

chain = chat_prompt | llm | output_parser

response = chain.invoke({})
print(response)

# Construct React Agent

In [ ]:
%pip install tavily-python 

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.chat_models import ChatMaritalk

tools = [TavilySearchResults(max_results=1)]

# Get the prompt from langchain hub
prompt = hub.pull("hwchase17/react")

llm = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

# Construct the React agent
agent = create_react_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "what is LangChain?"})

# Streaming

In [ ]:
from langchain_core.messages import HumanMessage

from langchain_community.chat_models import ChatMaritalk
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate

llm = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

output_parser = StrOutputParser()

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant! Your name is Bob."),
        ("human", "What is your name?")
    ]
)
print(chat_prompt)

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a helpful assistant! Your name is Bob."
    ),
    HumanMessage(
        content="What is your name?"
    )
]
print(messages)

chain = chat_prompt | llm | output_parser

for chunk in llm.stream(messages):
    print(chunk.content, end="", flush=True)

# Message Persistence

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.messages import HumanMessage

from langchain_community.chat_models import ChatMaritalk
from langchain_core.prompts.chat import ChatPromptTemplate

model = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant! Your name is Bob."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
print(prompt_template)


# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim. whats your name"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# Managing Conversation History

In [ ]:
%pip install trimmer
%pip install transformers

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.messages import AIMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.messages import HumanMessage

from langchain_community.chat_models import ChatMaritalk
from langchain_core.prompts.chat import ChatPromptTemplate


model = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
]

trimmer.invoke(messages)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage, trim_messages

from langchain_community.chat_models import ChatMaritalk
from langchain_core.prompts.chat import ChatPromptTemplate

model = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
    ]
)
print(prompt_template)


# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What was my previous question?"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
input_messages